In [17]:
import pandas as pd
import re
from nltk.corpus import stopwords
import pymorphy3
from tqdm.auto import tqdm

In [9]:
aviation_data = pd.read_csv('../data/raw/aviation.csv')
road_transport_data = pd.read_csv('../data/raw/road_transport.csv')

print(aviation_data.info())
print(road_transport_data.info())

/var/folders/bt/nbybvdjd4dl4j63pnps0226w0000gn/T/ipykernel_31214/36430761.py:1: DtypeWarning: Columns (21,22,24,26,28,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_data = pd.read_csv('../data/raw/aviation.csv')
/var/folders/bt/nbybvdjd4dl4j63pnps0226w0000gn/T/ipykernel_31214/36430761.py:2: DtypeWarning: Columns (9,22,23,28,29,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  road_transport_data = pd.read_csv('../data/raw/road_transport.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151222 entries, 0 to 151221
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            151222 non-null  int64  
 1   inner_type            151222 non-null  object 
 2   comments              151222 non-null  object 
 3   marked_as_ads         151222 non-null  int64  
 4   hash                  151221 non-null  object 
 5   type                  151222 non-null  object 
 6   push_subscription     143011 non-null  object 
 7   is_added_to_main_tab  151222 non-null  bool   
 8   attachments           151222 non-null  object 
 9   date                  151222 non-null  int64  
 10  edited                23884 non-null   float64
 11  from_id               151222 non-null  int64  
 12  id                    151222 non-null  int64  
 13  is_favorite           151222 non-null  bool   
 14  likes                 148889 non-null  object 
 15  

In [10]:
data = pd.concat([aviation_data, road_transport_data])

data = data[data['marked_as_ads'] == 0]

data = data[['text', 'y']]

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328652 entries, 0 to 180189
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    302089 non-null  object
 1   y       328652 non-null  object
dtypes: object(2)
memory usage: 7.5+ MB


In [ ]:
data.dropna(inplace=True)
data.drop_duplicates(keep='first', inplace=True)

y_map = {
    'aviation': 0,
    'road_transport': 1
}
data['y'] = data['y'].map(y_map)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 289198 entries, 0 to 180187
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    289198 non-null  object
 1   y       289198 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.6+ MB
None

Доли постов каждой категории: y
1    0.511601
0    0.488399
Name: proportion, dtype: float64


In [ ]:
morph = pymorphy3.MorphAnalyzer()
stopwords = set(stopwords.words('russian'))

garbage_list = [
    'http', 'https', 'vk', 'cc', 'com', 'ru', 'me', 'avtoobmen'
]
stopwords.update(garbage_list)

def preprocess_text(text):
    if not isinstance(text, str): 
        return ""

    text = text.lower()
    text = re.sub(r'(https?://\S+)|(www\.\S+)|(\w+\.(com|net|org|ru|me|cc|to)/\S+)', ' ', text)
    text = re.sub(r'\[(id|club)\d+\|.*?\]', '', text)
    text = re.sub(r'[^a-zа-яё\s]', ' ', text)

    words = text.split()

    lemmas = []
    for word in words:
        if len(word) > 1:
            lemma = morph.parse(word)[0].normal_form
            
            if lemma not in stopwords:
                lemmas.append(lemma)

    return " ".join(lemmas)


print("\nНачало предобработки текста...")

tqdm.pandas()
data['processed_text'] = data['text'].progress_apply(preprocess_text)

print("Предобработка текста завершена.")

display(data.head())


Начало предобработки текста...


100%|██████████| 289198/289198 [24:40<00:00, 195.34it/s] 


Предобработка текста завершена.


,text,y,processed_text
0,Что делаете на Новый год?\n\nКак насчёт смены ...,0,делать новый год насчёт смена привычный обстан...
1,"Для тех, кто часто летает, выпустили Бизнес-пр...",0,часто летать выпустить бизнес проездной абонем...
2,Декабрь — месяц больших покупок. Превращайте н...,0,декабрь месяц больший покупка превращать новог...
3,Провели матчдей «Зенит» - «Рубин» на «Газпром ...,0,провести матчдея зенит рубин газпром арена лаз...
4,Перелёт с нами — это важная деталь идеального ...,0,перелёт это важный деталь идеальный путешестви...


In [13]:
data.to_csv('../data/processed/processed_data.csv')

In [14]:
print(*data['processed_text'][0])

print('-' * 150)

print(*data['text'][0])

делать новый год насчёт смена привычный обстановка декабрь праздник новый город страна яркий впечатление фанат эстетика знать выбирать маршрут ваш следующий путешествие россия сочи москва санкт петербург новосибирск уфа самара калининград кировск апатит красноярск краснодар владивосток красноярск рубеж москва стамбул гонконг каир ереван минск пекин шанхай возвращаться россия стамбул москва бишкек москва ош красноярск самарканд санкт петербург ташкент москва вылет декабрь январь зависеть направление бронировать билет наш сайт спойлер включить уведомление следить наш лента готовить грандиозный декабрьский сюрприз ускорить публикация написать пометка хотеть пост очередь телеграмма барахолка авто москва канал барахолка авто москва диалог
------------------------------------------------------------------------------------------------------------------------------------------------------
Что делаете на Новый год?

Как насчёт смены привычной обстановки в декабре или в сам праздник? Новые горо